In [8]:
# from langchain.llms import OpenAI
from langchain_openai import OpenAI

In [6]:
import os
os.environ['OPENAI_API_KEY'] = ""

temperature value -> How creative we want out model to be <br>
It's betwen 0 and 1 <br>
0 -> model is playing safe and not taking any risks <br>
1 -> model is taking risk and might generate wrong results be very creative

In [9]:
llm = OpenAI(temperature = 0.6)

In [ ]:
text = "What is the capital of United States"
print(llm.invoke(text))

In [12]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''


In [30]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(repo_id = "google/flan-t5-large", model_kwargs = {'temperature':0.6, "max_length":64})

In [31]:
output = llm_huggingface.invoke(text)
print(output)

washington dc


In [32]:
output = llm_huggingface.invoke("Write a Poem about AI")
print(output)

AI is a machine that can do anything it wants to do. It can do anything it wants to do. It can do anything it wants to do. It can do anything it wants to do. It can do anything it wants to do. It can do anything it wants to do. It can do anything


## Prompt Templates and LLMChains

In [33]:
from langchain.prompts import PromptTemplate

In [35]:
prompt_template = PromptTemplate(
        input_variables=['country'],
        template="Tell me the capital of {country}"
    )
prompt_template.format(country="India")

'Tell me the capital of India'

In [36]:
from langchain.chains import LLMChain

In [ ]:
chain = LLMChain(llm = llm, prompt = prompt_template)
print(chain.run("United States"))

## Combining Multiple Chains Using Simple Sequential Chain

In [40]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

## Sequencial Chain

In [41]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [42]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [ ]:
chain({'country':"India"})

## ChatModels with Open AI

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [45]:
chatllm = ChatOpenAI(temperature = 0.6, model = 'gpt-3.5-turbo')

In [ ]:
chatllm([
    SystemMessage(content = "You are a comedian AI Assistant"),
    HumanMessage(content = "Please provide some comedy punchlines on AI.")
])

# Prompt Template + LLM + Output Parsers

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [48]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [49]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [50]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})